In [ ]:
import pandas as pd

1. Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.


In [ ]:
horse_data = pd.read_csv('horse_data.csv',  na_values = '?', usecols = [0, 1, 3, 4, 5, 6, 10, 22], header = None,
names = ['surgery?','Age','rectal temperature','pulse','respiratory rate','temperature of extremities','pain','outcome'])
horse_data

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


2.Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.


In [ ]:
df = horse_data
#методом unique проверены все столбцы-категории. Во всех, кроме "Age", есть NaN. В "Age" есть незаявленная категория 9.
df['outcome'].unique()

array([ 2.,  3.,  1., nan])

In [ ]:
#А в столбце Age есть 24 значения, не заявленные в Attribute Information. Это может быть ошибкой в данных, или в описании
df['Age'].value_counts()

1    276
9     24
Name: Age, dtype: int64

In [ ]:
df['surgery?'].value_counts()

1.0    180
2.0    119
Name: surgery?, dtype: int64

In [ ]:
df['temperature of extremities'].value_counts()

3.0    109
1.0     78
2.0     30
4.0     27
Name: temperature of extremities, dtype: int64

In [ ]:
df['pain'].value_counts()

3.0    67
2.0    59
5.0    42
4.0    39
1.0    38
Name: pain, dtype: int64

In [ ]:
df['outcome'].value_counts()

1.0    178
2.0     77
3.0     44
Name: outcome, dtype: int64

In [ ]:
#гугл говорит, что у молодых лошадей показатель пульса в 2 и более раз выше, чем у взрослых лошадей.
#Учитывая нормальные референсные значения пульса у лошадей, с поправкой на то, что лошади больны или в стрессе и их пульс еще выше,
#предположу, что "9" в столбце "Age" - это молодые лошади. Так что, допустим, "9" - не ошибка и не пропуск
df.groupby(['Age', 'surgery?'])['pulse'].median()

Age  surgery?
1    1.0          66.0
     2.0          54.0
9    1.0         129.0
     2.0          91.0
Name: pulse, dtype: float64

In [ ]:
df_number = df[['rectal temperature','pulse','respiratory rate']]
df_number.describe()

,rectal temperature,pulse,respiratory rate
count,240.000000,276.000000,242.000000
mean,38.167917,71.913043,30.417355
std,0.732289,28.630557,17.642231
min,35.400000,30.000000,8.000000
25%,37.800000,48.000000,18.500000
50%,38.200000,64.000000,24.500000
75%,38.500000,88.000000,36.000000
max,40.800000,184.000000,96.000000


In [ ]:
#межквартильный размах:
q1 = df['rectal temperature'].quantile(0.25)
q3 = df['rectal temperature'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (2.5 * iqr)
upper_bound = q3 + (2.5 * iqr)

#смотрю, что считается выбросами для температуры, если принимать такие lower_bound и upper_bound:
df.loc[(~df['rectal temperature'].between(lower_bound, upper_bound, inclusive = 'both')) & (~df['rectal temperature'].isna())]['rectal temperature']

44     35.4
54     40.3
91     40.3
141    36.0
259    40.8
Name: rectal temperature, dtype: float64

In [ ]:
#межквартильный размах и возможные выбросы для пульса:
q1 = df['pulse'].quantile(0.25)
q3 = df['pulse'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)
df.loc[(~df['pulse'].between(lower_bound, upper_bound, inclusive = 'both')) & (~df['pulse'].isna())]['pulse']

3      164.0
41     150.0
55     160.0
255    184.0
275    150.0
Name: pulse, dtype: float64

In [ ]:
#межквартильный размах и возможные выбросы для частоты дыхания:
q1 = df['respiratory rate'].quantile(0.25)
q3 = df['respiratory rate'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (3 * iqr)
upper_bound = q3 + (3 * iqr)
df.loc[(~df['respiratory rate'].between(lower_bound, upper_bound, inclusive = 'both')) & (~df['respiratory rate'].isna())]['respiratory rate']

#Лошади больны, так что могут иметь слишком заниженные/завышенные показатели температуры, пульса и частоты дыхания.
#Считаю, что эти значения находятся в рамках возможного. Не ошибки

39     96.0
106    96.0
186    90.0
269    90.0
Name: respiratory rate, dtype: float64

3.Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.


In [ ]:
#количество пропусков в стобце:
df.isna().mean() * 300

surgery?                       1.0
Age                            0.0
rectal temperature            60.0
pulse                         24.0
respiratory rate              58.0
temperature of extremities    56.0
pain                          55.0
outcome                        1.0
dtype: float64

In [ ]:
new_df = df.copy()
#в первом столбце всего один пропуск. Значение связано, возможно, с болью, и маловероятно с чем-то еще.
new_df['surgery?'] = new_df['surgery?'].fillna(df['surgery?'].mode()[0])

In [ ]:
#В horse_data.names сказано "In general, the more painful, the more likely it is to require surgery", т.е. есть взаимосвязь
print(df.groupby(['pain'])['surgery?'].median())
#думаю, оправданно заполнить столбец с оценкой боли именно на основе столбца surgery
print(df.groupby(['surgery?'])['pain'].median())


pain
1.0    2.0
2.0    1.0
3.0    1.0
4.0    1.0
5.0    1.0
Name: surgery?, dtype: float64
surgery?
1.0    3.0
2.0    2.0
Name: pain, dtype: float64


In [ ]:
#замена пропусков модой в столбце с оценкой боли:
new_df['pain'] = new_df['pain'].fillna(df.groupby(['surgery?'])['pain'].transform(lambda x: x.mode()[0]))

In [ ]:
#Исход болезни может быть связан, например, с болью
print(df.groupby(['pain'])['outcome'].median())

pain
1.0    1.0
2.0    1.0
3.0    1.0
4.0    2.0
5.0    2.0
Name: outcome, dtype: float64


In [ ]:
new_df['outcome'] = new_df['outcome'].fillna(df.groupby(['pain'])['outcome'].transform(lambda x: x.mode()[0]))

In [ ]:
#animals with painful lesions or suffering from circulatory shock may have an elevated heart rate
#проверим эту связь, учитывая, что оценка субъективна (на оценке боли "2" пропорциональность ломается).
#ЧСС главным образом зависит от того, жеребенок или взрослая это лошадь
print(df.groupby(['Age','pain'])['pulse'].median())

Age  pain
1    1.0      48.0
     2.0      72.0
     3.0      55.0
     4.0      77.5
     5.0      88.0
9    1.0      92.0
     2.0     124.0
     3.0     120.0
     4.0     128.0
     5.0     136.0
Name: pulse, dtype: float64


In [ ]:
new_df['pulse'] = new_df['pulse'].fillna(new_df.groupby(['Age','pain'])['pulse'].transform('median'))

In [ ]:
#гугл говорит, что частота дыхания у лошадей увеличивается при боли или повышении температуры. норма 8 - 14
#на оценке боли "2" пропорциональность не соответствует теории, как и в случае с ЧСС.
#Однако, сходится то, что ЧСС растет прямо пропорционально частоте дыхания. Видимо, дело в загадочной оценке "2"
print(df.groupby(['Age','pain'])['respiratory rate'].median())

Age  pain
1    1.0     20.0
     2.0     28.0
     3.0     24.0
     4.0     27.5
     5.0     28.0
9    1.0     28.0
     2.0     72.0
     3.0     40.0
     4.0     44.0
     5.0     49.0
Name: respiratory rate, dtype: float64


In [ ]:
new_df['respiratory rate'] = new_df['respiratory rate'].fillna(new_df.groupby(['Age','pain'])['respiratory rate'].transform('median'))

In [ ]:
#найти зависимость столбца с температурой от других не удалось. Но СКО мало, и, скорее всего, стоит заменить пропуски средним
new_df['rectal temperature'] = new_df['rectal temperature'].fillna(df['rectal temperature'].mean())

In [ ]:
#cool to cold extremities indicate possible shock
#hot extremities should correlate with an elevated rectal temp.
#1 = Normal
#2 = Warm
#3 = Cool
#4 = Cold
#Я не увидела явных корреляций или зависимостей, ни с болью, ни с температурой. Пусть в основном у них будут прохладные конечности
print(df.groupby(['temperature of extremities','pain'])['rectal temperature'].median())

temperature of extremities  pain
1.0                         1.0     38.00
                            2.0     37.90
                            3.0     38.20
                            4.0     38.10
                            5.0     38.30
2.0                         1.0     38.30
                            2.0     38.60
                            3.0     38.00
                            4.0     38.25
                            5.0       NaN
3.0                         1.0     38.20
                            2.0     38.65
                            3.0     38.10
                            4.0     37.90
                            5.0     38.00
4.0                         2.0     38.50
                            3.0     38.30
                            4.0     38.70
                            5.0     38.60
Name: rectal temperature, dtype: float64


In [ ]:
new_df['temperature of extremities'] = new_df['temperature of extremities'].fillna(df['temperature of extremities'].mode()[0])

In [ ]:
#пропусков больше нет:
new_df.isna().mean() * 300

surgery?                      0.0
Age                           0.0
rectal temperature            0.0
pulse                         0.0
respiratory rate              0.0
temperature of extremities    0.0
pain                          0.0
outcome                       0.0
dtype: float64

In [ ]:
new_df[['rectal temperature','pulse','respiratory rate']].describe()

,rectal temperature,pulse,respiratory rate
count,300.000000,300.000000,300.000000
mean,38.167917,72.213333,29.940000
std,0.654705,27.997811,16.530798
min,35.400000,30.000000,8.000000
25%,37.900000,48.000000,20.000000
50%,38.167917,64.000000,25.500000
75%,38.500000,88.000000,36.000000
max,40.800000,184.000000,96.000000


In [ ]:
df_number.describe()

,rectal temperature,pulse,respiratory rate
count,240.000000,276.000000,242.000000
mean,38.167917,71.913043,30.417355
std,0.732289,28.630557,17.642231
min,35.400000,30.000000,8.000000
25%,37.800000,48.000000,18.500000
50%,38.200000,64.000000,24.500000
75%,38.500000,88.000000,36.000000
max,40.800000,184.000000,96.000000
